In [2]:
from langchain_docling import DoclingLoader
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

/home/aasmund_mjos/Downloads/sintef/mrst-rag-project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
start_link = "https://www.sintef.no/projectweb/mrst/publications/"

response = requests.get(start_link)
soup = BeautifulSoup(response.content, 'html.parser')

documents = []

links = set()

for item_1 in soup.find_all("li"):
    for item_2 in item_1.find_all("a"):
        link = item_2.get("href")
        if "publications" in link:
            links.add("https://www.sintef.no" + link)

links.discard(start_link)

link_to_file_type = {}
file_type_to_link = {}

for link in links:

    file_type = link[len(start_link):-1]

    link_to_file_type[link] = file_type
    file_type_to_link[file_type] = link

ntnu_links = []

for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    documents = []
    for section in soup.find_all('div', class_ = "rich-text-field"):
        for article in section.find_all("li"):
            for link_box in article.find_all('a'):
                paper_link = link_box.get('href')
                if paper_link != None:
                    if 'folk.ntnu' in paper_link:
                        ntnu_links.append((paper_link, link))

print("Done gathering folk.ntnu papers")

n = len(ntnu_links)

for i,(paper_link, link) in enumerate(ntnu_links):
    print(f"Loading document {i+1} out of {n}", end = "\r")
    loader = DoclingLoader(file_path=paper_link)
    docs = loader.load()
    documents.append(docs)

Done gathering folk.ntnu papers


Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (2239 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (1223 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (2369 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (2369 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (775 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (1019 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from typing import List

def split_by_more(string: str, chars: List[str] = []):
    if chars == []:
        return string.split()
    else:
        for method, next_method in zip(chars, chars[1:]):
            string = next_method.join(string.split(method))
        return string.split(chars[-1])

In [60]:
titles = []
content = []
metadata = []
source = []
chunk = []
authors = []
file_type = []

for (paper_link, link, t, a), paper_documents in zip(ntnu_links, documents):
    if len(paper_documents):
        for i,paper_doc in enumerate(paper_documents):

            m = paper_doc.metadata.get('dl_meta')
            titles.append(t)
            authors.append(a)
            metadata.append(m)
            source.append(paper_link)
            file_type.append(link)
            chunk.append(i)
            content.append(paper_doc.page_content)

df = pd.DataFrame({"authors": authors, "content": content, "title": titles, "metadata": metadata, "file_type": file_type, "chunk": chunk, "source": source})
df.to_pickle("folk_ntnu_df.pkl")

In [53]:
start_link = "https://www.sintef.no/projectweb/mrst/publications/"

response = requests.get(start_link)
soup = BeautifulSoup(response.content, 'html.parser')

links = set()

for item_1 in soup.find_all("li"):
    for item_2 in item_1.find_all("a"):
        link = item_2.get("href")
        if "publications" in link:
            links.add("https://www.sintef.no" + link)

links.discard(start_link)

link_to_file_type = {}
file_type_to_link = {}

for link in links:

    file_type = link[len(start_link):-1]

    link_to_file_type[link] = file_type
    file_type_to_link[file_type] = link

ntnu_links = []

for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    for section in soup.find_all('div', class_ = "rich-text-field"):
        for article in section.find_all("li"):
            for link_box in article.find_all('a'):
                paper_link = link_box.get('href')
                if paper_link != None:
                    if 'folk.ntnu' in paper_link:
                        title = link_box.get_text()
                        text = article.get_text()
                        index = text.find(title)
                        name_text = text[:index]
                        authors = split_by_more(name_text, [", and", ", "])
                        ntnu_links.append((paper_link, link, title, authors))

print("Done gathering folk.ntnu papers")

Done gathering folk.ntnu papers


In [6]:
df = pd.read_pickle('../folk_ntnu_df.pkl')
df

,authors,content,title,metadata,file_type,chunk,source
0,"[T. S. Mykkeltvedt, X. Raynaud, K.-A. Lie. ]",Fully Implicit Higher-Order Schemes Applied to...,Fully implicit higher-order schemes applied to...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,0,https://folk.ntnu.no/andreas/papers/comg-poly-...
1,"[T. S. Mykkeltvedt, X. Raynaud, K.-A. Lie. ]",Fully Implicit Higher-Order Schemes Applied to...,Fully implicit higher-order schemes applied to...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,1,https://folk.ntnu.no/andreas/papers/comg-poly-...
2,"[T. S. Mykkeltvedt, X. Raynaud, K.-A. Lie. ]",Fully Implicit Higher-Order Schemes Applied to...,Fully implicit higher-order schemes applied to...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,2,https://folk.ntnu.no/andreas/papers/comg-poly-...
3,"[T. S. Mykkeltvedt, X. Raynaud, K.-A. Lie. ]",1 Introduction\nIn many oil recovery processes...,Fully implicit higher-order schemes applied to...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,3,https://folk.ntnu.no/andreas/papers/comg-poly-...
4,"[T. S. Mykkeltvedt, X. Raynaud, K.-A. Lie. ]",1 Introduction\nAs an example of water-based E...,Fully implicit higher-order schemes applied to...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,4,https://folk.ntnu.no/andreas/papers/comg-poly-...
...,...,...,...,...,...,...,...
2811,"[A. R. Syversveen, H. M. Nilsen, K.-A. Lie, J....",6 Concluding remarks\nCO2 is affected by top-r...,A study on how top-surface morphology influenc...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,34,http://folk.ntnu.no/andreas/papers/geostats-20...
2812,"[A. R. Syversveen, H. M. Nilsen, K.-A. Lie, J....","References\n1. Abrahamsen, P., Hauge, R., Hegg...",A study on how top-surface morphology influenc...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,35,http://folk.ntnu.no/andreas/papers/geostats-20...
2813,"[A. R. Syversveen, H. M. Nilsen, K.-A. Lie, J....","References\n3. Class, H., Ebigbo, A., Helmig, ...",A study on how top-surface morphology influenc...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,36,http://folk.ntnu.no/andreas/papers/geostats-20...
2814,"[A. R. Syversveen, H. M. Nilsen, K.-A. Lie, J....","References\n4. Hollund, K., Mostad, P., Nielse...",A study on how top-surface morphology influenc...,{'schema_name': 'docling_core.transforms.chunk...,https://www.sintef.no/projectweb/mrst/publicat...,37,http://folk.ntnu.no/andreas/papers/geostats-20...
